## 라이브러리 호출 및 DPU 환경 설정

In [1]:
import os
import sys
import time
import cv2
import numpy as np
import threading
import keyboard
import spidev
from datetime import datetime
from IPython.display import clear_output, display
import ipywidgets as widgets
from pynq import Overlay, MMIO
from pynq_dpu import DpuOverlay
import matplotlib.pyplot as plt

# Load the overlay
overlay = DpuOverlay("../dpu/dpu.bit")

## 모터 초기설정

In [2]:
# 모터 설정
motor_0_address = 0x00A0000000
motor_1_address = 0x00A0010000
motor_2_address = 0x00A0020000
motor_3_address = 0x00A0030000
motor_4_address = 0x00A0040000
motor_5_address = 0x00A0050000

address_range = 0x10000

motor_0 = MMIO(motor_0_address , address_range)
motor_1 = MMIO(motor_1_address , address_range) # CHANGE 
motor_2 = MMIO(motor_2_address , address_range)
motor_3 = MMIO(motor_3_address , address_range)
motor_4 = MMIO(motor_4_address , address_range) # CHANGE
motor_5 = MMIO(motor_5_address , address_range)

In [3]:
# 모터 기본 설정
# size = 2ms ==3.33ns x 600600.600..... 
# about 600600
size = 600600
motor_0_percent = 50/100
motor_1_percent = 80/100
motor_2_percent = 50/100
motor_3_percent = 50/100
motor_4_percent = 50/100
motor_5_percent = 50/100

motor_0_duty = size * motor_0_percent
motor_1_duty = size * motor_1_percent
motor_2_duty = size * motor_2_percent
motor_3_duty = size * motor_3_percent
motor_4_duty = size * motor_4_percent
motor_5_duty = size * motor_5_percent
print(int(motor_0_duty))


motor_0.write(0x00,size)           # size
motor_0.write(0x04,int(motor_0_duty))   # DUTY
motor_0.write(0x08,0)              # valid

motor_1.write(0x00,size)  # size
motor_1.write(0x04,int(motor_1_duty))   # DUTY
motor_1.write(0x08,0)       # valid

motor_2.write(0x00,size)  # size
motor_2.write(0x04,int(motor_2_duty))   # DUTY
motor_2.write(0x08,0)       # valid

motor_3.write(0x00,size)  # size
motor_3.write(0x04,int(motor_3_duty))   # DUTY
motor_3.write(0x08,0)       # valid

motor_4.write(0x00,size)  # size
motor_4.write(0x04,int(motor_4_duty))   # DUTY
motor_4.write(0x08,0)       # valid

motor_5.write(0x00,size)  # size
motor_5.write(0x04,int(motor_5_duty))   # DUTY
motor_5.write(0x08,0)       # valid

300300


## 모터 테스트

In [4]:
# motor 0 rear_right_front
motor_0.write(0x08,1)             
motor_1.write(0x08,0)              

motor_2.write(0x08,0)           
motor_3.write(0x08,0)         

motor_4.write(0x08,0)             
motor_5.write(0x08,0)             

In [ ]:
# motor 1 steering_left
motor_0.write(0x08,0)             
motor_1.write(0x08,1)            

motor_2.write(0x08,0)              
motor_3.write(0x08,0)          

motor_4.write(0x08,0)              
motor_5.write(0x08,0)              

In [6]:
# motor 2 rear_left_back
motor_0.write(0x08,0)              
motor_1.write(0x08,0)          

motor_2.write(0x08,1)             
motor_3.write(0x08,0)             

motor_4.write(0x08,0)          
motor_5.write(0x08,0)    

In [7]:
# motor 3 rear_left_front
motor_0.write(0x08,0)            
motor_1.write(0x08,0)            

motor_2.write(0x08,0)           
motor_3.write(0x08,1)     

motor_4.write(0x08,0)           
motor_5.write(0x08,0)     

In [8]:
# motor 4  rear_right_back
motor_0.write(0x08,0)          
motor_1.write(0x08,0)         

motor_2.write(0x08,0)       
motor_3.write(0x08,0)            

motor_4.write(0x08,1)           
motor_5.write(0x08,0)  

In [ ]:
# motor 5 steering_right 
motor_0.write(0x08,0)              
motor_1.write(0x08,0)           

motor_2.write(0x08,0)           
motor_3.write(0x08,0)           

motor_4.write(0x08,0)        
motor_5.write(0x08,1)      

In [10]:
# OFF
motor_5.write(0x08,0)              # valid  steering_right
motor_1.write(0x08,0)              # valid  steering_left
motor_2.write(0x08,0)              # valid rear_left_back
motor_3.write(0x08,0)              # valid rear_left_front
motor_4.write(0x08,0)              # valid rear_right_back
motor_0.write(0x08,0)              # valid rear_right_front

## SPI TEST

In [13]:
import spidev
import time
import sys

# SPI 설정
spi0 = spidev.SpiDev()

spi0.open(0, 0) 

# SPI 속도 설정 (최대 20MHz, Pmod AD1은 최대 1MHz 권장)
spi0.max_speed_hz = 20000000

# SPI 모드 설정 (모드 0)
spi0.mode = 0b00

def read_adc(spi):
    adc_response = spi.xfer2([0x00, 0x00])
    adc_value = ((adc_response[0] & 0x0F) << 8) | adc_response[1]
    return adc_value

try:
    while True:
        # D0 채널에서 데이터 읽기
        adc_value0 = read_adc(spi0)
        # 출력 내용을 덮어쓰기
        #sys.stdout.write(f"\rD0 채널 값: {adc_value0*3.3/4096}")
        sys.stdout.write(f"\rD0 채널 값: {adc_value0}")
        sys.stdout.flush()
        time.sleep(0.5)
except KeyboardInterrupt:
    spi0.close()
##우측 240
#좌측 990

D0 채널 값: 579

## Main 코드

In [ ]:
# SPI 설정
spi0 = spidev.SpiDev()
spi0.open(0, 0) 
spi0.max_speed_hz = 20000000
spi0.mode = 0b00

DATA_COLLECTION_PATH = "./collected_data"

def create_data_folder(base_path):
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder_path = os.path.join(base_path, current_time)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

class RobotController:
    def __init__(self):
        self.frame_num = 0
        self.data_folder = None
        self.size = 600600  # 2ms
        
        self.left_speed = 0
        self.right_speed = 0
        self.steering_angle = 0
        
        # 조향 관련 변수들
        self.current_duty = self.size // 2  # 현재 duty 값
        self.min_duty = self.size // 2      # 최소 duty 값 (50%)
        self.max_duty = int(self.size * 0.8)  # 최대 duty 값 (80%)
        self.duty_step = int(self.size * 0.02)  # duty 증가량 (2%)
        self.last_steering_time = time.time()  # 마지막 조향 시간
        
        # 가변저항 최대/최소값
        self.resistance_most_left = 990 
        self.resistance_most_right = 240

        # 쓰레드 종료 플래그
        self.stop_flag = False  # 추가

        self.init_motors()
    
    def init_motors(self):
        motors = [motor_0, motor_1, motor_2, motor_3, motor_4, motor_5]
        for motor in motors:
            motor.write(0x00, self.size)     # size
            motor.write(0x04, self.min_duty) # 초기 duty 50%
            motor.write(0x08, 0)            # valid=0

    def map_value(self, x, in_min, in_max, out_min, out_max):
        """
        x를 in_min~in_max 범위에서 out_min~out_max 범위로 매핑
        """
        if x <= in_min:
            return out_max
        elif x >= in_max:
            return out_min
        else:
            # in_min과 in_max 사이일 경우, x가 커질수록 결과가 선형적으로 감소하도록 계산
            return (in_max - x) * (out_max - out_min) / (in_max - in_min) + out_min

    def read_adc(self, spi):
        adc_response = spi.xfer2([0x00, 0x00])
        adc_value = ((adc_response[0] & 0x0F) << 8) | adc_response[1]
        return adc_value

    def right(self):
        current_time = time.time()
        if current_time - self.last_steering_time > 0.05:
            self.current_duty = min(self.max_duty, self.current_duty + self.duty_step)
            self.last_steering_time = current_time
            
        motor_1.write(0x08, 0)  # steering_left
        motor_5.write(0x08, 1)  # steering_right
        motor_5.write(0x04, self.current_duty)
        
    def left(self):
        current_time = time.time()
        if current_time - self.last_steering_time > 0.05:
            self.current_duty = min(self.max_duty, self.current_duty + self.duty_step)
            self.last_steering_time = current_time

        motor_5.write(0x08, 0)  # steering_left
        motor_1.write(0x08, 1)  # steering_right
        motor_1.write(0x04, self.current_duty)
        
    def stay(self):
        self.current_duty = self.min_duty
        motor_5.write(0x08, 0)
        motor_1.write(0x08, 0)
        motor_5.write(0x04, self.current_duty)
        motor_1.write(0x04, self.current_duty)
        
    def set_left_speed(self, speed):
        duty_percent = abs(speed) / 100
        duty = int(self.size * duty_percent)
        
        motor_2.write(0x04, duty)
        motor_3.write(0x04, duty)
        
        if speed > 0:
            motor_2.write(0x08, 0)
            motor_3.write(0x08, 1)
        else:
            motor_2.write(0x08, 1)
            motor_3.write(0x08, 0)
            
    def set_right_speed(self, speed):
        duty_percent = abs(speed) / 100
        duty = int(self.size * duty_percent)
       
        motor_4.write(0x04, duty)
        motor_0.write(0x04, duty)
        if speed > 0:
            motor_4.write(0x08, 0)
            motor_0.write(0x08, 1)
        else:
            motor_4.write(0x08, 1)
            motor_0.write(0x08, 0)
            
    def capture_frame(self, frame):
        # 폴더가 없을 경우 폴더 생성
        if self.data_folder is None:
            self.data_folder = create_data_folder(DATA_COLLECTION_PATH)

        file_name = os.path.join(
            self.data_folder,
            f"{self.frame_num}_speed:{self.left_speed}_steer:{self.steering_angle}.png"
        )
        cv2.imwrite(file_name, frame)
        print(f"프레임 저장: {file_name}")
        self.frame_num += 1

    def control_motors(self):
        mapped_resistance = self.map_value(
            self.read_adc(spi0),
            self.resistance_most_right,
            self.resistance_most_left,
            -20, 20
        )

        if abs(mapped_resistance - self.steering_angle) < 0.5:
            self.stay()
        elif mapped_resistance > self.steering_angle:
            self.left()
        else:
            self.right()

        self.set_left_speed(self.left_speed)
        self.set_right_speed(self.right_speed)
        
    def process_and_control(self, frame):
        """프레임 처리 로직이 있으면 여기에 추가"""
        return frame

    # -------------------------------------------------
    # (1) 키보드 전담 쓰레드 함수 - 최소 수정으로 추가
    # -------------------------------------------------
    def keyboard_thread_func(self):
        """
        기존 run() 내부 while문에 있던 키보드 입력 로직만 따로 분리.
        모터 제어(control_motors)까지 포함시켜, 키 입력 시 즉각 반응.
        """
        while not self.stop_flag:
            if keyboard.is_pressed('w'):
                self.left_speed += 0.5
                self.right_speed += 0.5
                print('speed: ', self.left_speed)
            elif keyboard.is_pressed('s'):
                self.left_speed -= 0.5
                self.right_speed -= 0.5
                print('speed: ', self.left_speed)
            elif keyboard.is_pressed('a'):
                self.steering_angle -= 0.5
                print('steering: ', self.steering_angle)
            elif keyboard.is_pressed('d'):
                self.steering_angle += 0.5
                print('steering: ', self.steering_angle)
            elif keyboard.is_pressed('r'):
                self.left_speed = 0
                self.right_speed = 0
                self.steering_angle = 0
            elif keyboard.is_pressed('c'):
                if getattr(self, "current_frame", None) is not None:
                    self.capture_frame(self.current_frame)
            elif keyboard.is_pressed('q'):
                print("프로그램 종료 (q키 입력)")
                self.stop_flag = True
                break

            # 속도 및 조향값 제한
            self.left_speed   = max(-100, min(100, self.left_speed))
            self.right_speed  = max(-100, min(100, self.right_speed))
            self.steering_angle = max(-20, min(20, self.steering_angle))

            # 모터 제어
            self.control_motors()

            # 너무 빠른 반복을 막기 위해 살짝 sleep
            time.sleep(0.01)

    # -------------------------------------------------
    # (2) 메인(카메라) 루프 
    # -------------------------------------------------
    def run(self, camera_index=0):
        cap = cv2.VideoCapture(camera_index)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 256)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 256)

        if not cap.isOpened():
            print("카메라(또는 영상 파일)를 열 수 없습니다.")
            return

        image_widget = widgets.Image(format='jpeg')

        # 키보드 전담 쓰레드 시작
        kb_thread = threading.Thread(target=self.keyboard_thread_func, daemon=True)
        kb_thread.start()

        try:
            while not self.stop_flag:
                ret, frame = cap.read()
                if not ret:
                    print("프레임을 읽을 수 없습니다. 영상이 끝났거나 오류가 발생했습니다.")
                    break

                # 메인 루프에서 읽어온 frame을 키보드 쓰레드가 capture_frame()에 쓸 수 있게 공유
                self.current_frame = frame

                processed_image = self.process_and_control(frame)
                ret_jpeg, jpeg = cv2.imencode('.jpeg', processed_image)
                if ret_jpeg:
                    image_widget.value = jpeg.tobytes()
                    display(image_widget)
                    # 카메라 디버깅을 위한 SW delay 추가
                    time.sleep(0.3)  # <-- main 루프에서의 딜레이
                    clear_output(wait=True)
                else:
                    print("이미지 인코딩 실패")

        except KeyboardInterrupt:
            print("사용자 중지 (Ctrl+C)")
        finally:
            self.stop_flag = True  # 쓰레드 종료 신호
            kb_thread.join()       # 키보드 쓰레드 종료 대기
            cap.release()
            self.cleanup()

    def cleanup(self):
        # 모터 정지
        motors = [motor_0, motor_1, motor_2, motor_3, motor_4, motor_5]
        for motor in motors:
            motor.write(0x00, 0)
            motor.write(0x04, 0)
            motor.write(0x08, 0)
        
        cv2.destroyAllWindows()
        clear_output(wait=True)
        print("모터 정지 및 종료.")

In [18]:
# 실행
controller = RobotController()
controller.run(camera_index=0)

모터 정지 및 종료.
